In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.endpoints import franchisehistory
from nba_api.stats.static import teams
from nba_api.stats.endpoints import playbyplayv2
import pandas as pd
import re



In [4]:
pattern = r"^(\d{4})-(\d{2})$"

def valid_seasons(str):
    match = re.match(pattern, str)
    if not match:
        return False
    
    start_year = int(match.group(1))
    end_year = int(match.group(2))

    expected_end = (start_year + 1) % 100
    if end_year != expected_end:
        return False
    all_seasons = leaguedashteamstats.LeagueDashTeamStats().available_seasons
    return str in all_seasons

In [15]:


franchises = franchisehistory.FranchiseHistory().get_data_frames()[0]
franchises['START_YEAR'] = pd.to_numeric(franchises['START_YEAR'])
franchises['END_YEAR'] = pd.to_numeric(franchises['END_YEAR'])


filtered = franchises[franchises['END_YEAR'] >= 1996]


nba_teams = pd.DataFrame(teams.get_teams())


merged = filtered.merge(nba_teams, left_on="TEAM_ID", right_on="id", how="left")


team_abbrs = merged['abbreviation'].dropna().unique().tolist()
team_abbrs.sort()

print(f"Teams from 1996-97 to present ({len(team_abbrs)} teams):")
print(team_abbrs)


Teams from 1996-97 to present (30 teams):
['ATL', 'BKN', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHX', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']


In [ ]:
print("Choose a team:")
for i, t in enumerate(team_abbrs, start=1):
    print(f"{i}. {t}")

choice = int(input("Enter number: "))
team = team_abbrs[choice-1]

print(f"You picked {team}")

Choose a team:
1. ATL
2. BKN
3. BOS
4. CHA
5. CHI
6. CLE
7. DAL
8. DEN
9. DET
10. GSW
11. HOU
12. IND
13. LAC
14. LAL
15. MEM
16. MIA
17. MIL
18. MIN
19. NOP
20. NYK
21. OKC
22. ORL
23. PHI
24. PHX
25. POR
26. SAC
27. SAS
28. TOR
29. UTA
30. WAS


In [ ]:
games = leaguegamefinder.LeagueGameFinder(season_nullable='2022-23').get_data_frames()[0]
team1 = "GSW"  # Golden State Warriors
team2 = "BOS"  # Boston Celtics

matchups = games[games['MATCHUP'].str.contains(team1) & games['MATCHUP'].str.contains(team2)]

pd.set_option("display.max_columns", None)
display(matchups[['GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL']])


game_id = matchups.iloc[0]['GAME_ID']  # for example, first one
pbp = playbyplayv2.PlayByPlayV2(game_id=game_id).get_data_frames()[0]
